# Show inside pickle

In [1]:
import dill as pickle
import sys
sys.path.append("/home/sina/env_prediction_project/Trajectron-plus-plus/trajectron")

# Open and load the .pkl file
with open("/home/sina/env_prediction_project/Trajectron-plus-plus/experiments/processed/eth_train.pkl", "rb") as file:
    data = pickle.load(file)

# Print the attributes of the data object
for attribute in dir(data):
    if not attribute.startswith('__'):
        print(attribute)
        try:
            value = getattr(data, attribute)
            print(f"{attribute}: {value}")
        except:
            None

NodeType
NodeType: [PEDESTRIAN]
_scenes_resample_prop
_scenes_resample_prop: None
attention_radius
attention_radius: {(PEDESTRIAN, PEDESTRIAN): 3.0}
get_edge_types
get_edge_types: <bound method Environment.get_edge_types of <environment.environment.Environment object at 0x7f1c29f1db38>>
get_standardize_params
get_standardize_params: <bound method Environment.get_standardize_params of <environment.environment.Environment object at 0x7f1c29f1db38>>
node_type_list
node_type_list: ['PEDESTRIAN']
robot_type
robot_type: None
scenes
scenes: [Scene: Duration: 284.40000000000003s, Nodes: 125, Map: No., Scene: Duration: 172.8s, Nodes: 364, Map: No., Scene: Duration: 336.40000000000003s, Nodes: 171, Map: No., Scene: Duration: 142.0s, Nodes: 373, Map: No., Scene: Duration: 576.0s, Nodes: 310, Map: No., Scene: Duration: 241.20000000000002s, Nodes: 105, Map: No., Scene: Duration: 237.60000000000002s, Nodes: 96, Map: No.]
scenes_resample_prop
standardization
standardization: {'PEDESTRIAN': {'position

# Show inside pytorch model

In [ ]:
import torch

model_data = torch.load("/home/sina/env_prediction_project/Trajectron-plus-plus/experiments/pedestrians/models/models_06_Mar_2025_15_57_30_eth_vel_ar3/model_registrar-32.pt")
print(type(model_data))

for name, module in model_data.items():
    print(f"Module name: {name}")
    print(module)

<class 'torch.nn.modules.container.ModuleDict'>
Module name: PEDESTRIAN/node_history_encoder
LSTM(6, 32, batch_first=True)
Module name: PEDESTRIAN/node_future_encoder
LSTM(2, 32, batch_first=True, bidirectional=True)
Module name: PEDESTRIAN/node_future_encoder/initial_h
Linear(in_features=6, out_features=32, bias=True)
Module name: PEDESTRIAN/node_future_encoder/initial_c
Linear(in_features=6, out_features=32, bias=True)
Module name: PEDESTRIAN/edge_influence_encoder
AdditiveAttention(
  (w1): Linear(in_features=32, out_features=32, bias=False)
  (w2): Linear(in_features=32, out_features=32, bias=False)
  (v): Linear(in_features=32, out_features=1, bias=False)
)
Module name: PEDESTRIAN/p_z_x
Linear(in_features=64, out_features=32, bias=True)
Module name: PEDESTRIAN/hx_to_z
Linear(in_features=32, out_features=25, bias=True)
Module name: PEDESTRIAN/hxy_to_z
Linear(in_features=192, out_features=25, bias=True)
Module name: PEDESTRIAN/decoder/state_action
Sequential(
  (0): Linear(in_featur

# Graphically view inside .pt file

In [12]:
!pip install netron
import netron

model_path = "/home/sina/env_prediction_project/Trajectron-plus-plus/experiments/pedestrians/models/models_06_Mar_2025_15_57_30_eth_vel_ar3/model_registrar-32.pt" 
netron.start(model_path)

Serving '/home/sina/env_prediction_project/Trajectron-plus-plus/experiments/pedestrians/models/models_06_Mar_2025_15_57_30_eth_vel_ar3/model_registrar-32.pt' at http://localhost:24050


('localhost', 24050)